In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

def generate_cities(n, Xmin, Xmax, Ymin, Ymax):
    cities = np.zeros((n, 2))
    for i in range(n):
        cities[i][0] = np.random.randint(Xmin, Xmax)
        cities[i][1] = np.random.randint(Ymin, Ymax)
    return cities

def problem(order, cities):
    def distance(P1, P2):
        return np.sqrt((P1[0] - P2[0])**2 + (P1[1] - P2[1])**2)
    
    total_distance = 0
    for i in range(1, len(order)):
        total_distance += distance(cities[order[i-1]], cities[order[i]])
    total_distance += distance(cities[order[-1]], cities[order[0]])
    return total_distance

def selection(pop, scores, k=3):
    selection_ix = random.randint(0, len(pop) - 1)
    for ix in random.sample(range(len(pop)), k):
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def ordered_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end+1] = parent1[start:end+1]
    ptr = (end + 1) % size
    for city in parent2:
        if city not in child:
            child[ptr] = city
            ptr = (ptr + 1) % size
    return child

def mutation(child, r_mut):
    for i in range(len(child)):
        if random.random() < r_mut:
            j = random.randint(0, len(child) - 1)
            child[i], child[j] = child[j], child[i]
    return child

def plot_path(cities, order, generation, score):
    plt.clf()
    x = [cities[i][0] for i in order]
    y = [cities[i][1] for i in order]
    x.append(x[0])  
    y.append(y[0])
    
    plt.plot(x, y, 'bo-', label=f"Generation {generation}, Distance: {score:.2f}")
    plt.scatter(x, y, c='red')
    for i, (xi, yi) in enumerate(zip(x, y)):
        plt.text(xi, yi, str(i), fontsize=12)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title(f"Best Path (Generation {generation})")
    plt.legend()

def genetic_algorithm(problem, cities, order, n_iter, n_pop, r_mut, tolerance=0.000001, elite_percent=0.3):
    pop = [random.sample(order, len(order)) for _ in range(n_pop)]
    
    best, best_eval = pop[0], problem(pop[0], cities)
    
    frames = []
    
    no_improvement_count = 0
    previous_best_eval = best_eval
    
    n_elite = int(elite_percent * n_pop)
    
    for gen in range(n_iter):
        scores = [problem(d, cities) for d in pop]
        
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(f"> Generation {gen}, new best f({pop[i]}) = {scores[i]}")
        
        frames.append((cities, best.copy(), gen, best_eval))
        
        if abs(previous_best_eval - best_eval) < tolerance:
            no_improvement_count += 1
        else:
            no_improvement_count = 0
        previous_best_eval = best_eval
        
        if no_improvement_count >= 70:
            print(f"Early stopping at generation {gen} (no improvement for 70 iterations).")
            break
        
        sorted_pop = [x for _, x in sorted(zip(scores, pop), key=lambda pair: pair[0])]
        elites = sorted_pop[:n_elite]
        
        selected = [selection(pop, scores) for _ in range(n_pop - n_elite)]
        
        if len(selected) % 2 != 0:
            selected.append(selected[-1])  
        children = []
        
        for i in range(0, len(selected), 2):
            p1, p2 = selected[i], selected[i + 1]
            child1 = ordered_crossover(p1, p2)
            child2 = ordered_crossover(p2, p1)
            child1 = mutation(child1, r_mut)
            child2 = mutation(child2, r_mut)
            children.append(child1)
            children.append(child2)
        
        pop = elites + children
    
    fig, ax = plt.subplots()
    
    def init():
        ax.clear()
    
    def update(frame):
        cities, order, gen, score = frame
        ax.clear()
        plot_path(cities, order, gen, score)
    
    ani = FuncAnimation(fig, update, frames=frames, init_func=init, repeat=False, interval=500)
    
    ani.save("tsp_animation.gif", writer=PillowWriter(fps=8))
    print("Animation saved as tsp_animation.gif")
    
    plt.show()  
    return [best, best_eval]

n_cities = 30
cities = generate_cities(n_cities, -1000, 1000, -1000, 1000)
initial_order = list(range(n_cities))

best, score = genetic_algorithm(problem, cities, initial_order, n_iter=1000, n_pop=1000, r_mut=0.1, elite_percent=0.4)
print("Best solution:", best)
print("Best score:", score)

> Generation 0, new best f([15, 17, 4, 8, 3, 16, 21, 29, 12, 14, 5, 2, 13, 23, 10, 18, 25, 24, 0, 28, 7, 9, 6, 11, 27, 26, 19, 1, 22, 20]) = 29255.97164704774
> Generation 0, new best f([24, 6, 21, 29, 7, 2, 14, 3, 20, 25, 28, 27, 19, 13, 17, 9, 18, 8, 15, 5, 16, 1, 4, 23, 10, 12, 11, 0, 22, 26]) = 29248.03091934066
> Generation 0, new best f([19, 9, 23, 0, 16, 5, 13, 28, 3, 29, 6, 7, 1, 25, 10, 14, 22, 18, 12, 17, 4, 26, 15, 2, 24, 27, 8, 21, 20, 11]) = 23097.443116012997
> Generation 0, new best f([27, 24, 22, 18, 23, 0, 19, 2, 5, 4, 17, 9, 20, 12, 29, 15, 25, 16, 6, 21, 28, 3, 10, 13, 7, 11, 26, 1, 14, 8]) = 23082.88215889933
> Generation 0, new best f([20, 22, 19, 8, 29, 15, 7, 14, 6, 27, 18, 0, 24, 16, 17, 23, 3, 1, 4, 5, 10, 28, 12, 21, 25, 13, 2, 9, 26, 11]) = 22170.493051476846
> Generation 0, new best f([29, 12, 21, 10, 18, 19, 22, 24, 16, 13, 7, 20, 27, 3, 25, 8, 1, 15, 5, 23, 14, 17, 0, 26, 9, 2, 11, 6, 4, 28]) = 21641.593756405164
> Generation 1, new best f([24, 23, 20, 6, 

KeyboardInterrupt: 